# Module 1: Foundations of Bayesian Thinking for Trading\n\n## Opening Hook: The 2020 Oil Price Anomaly\n\nOn April 20, 2020, WTI crude oil futures went negative for the first time in history, reaching -$37.63 per barrel. Traditional statistical models failed to predict or even consider this possibility. But what if you had a framework that could:\n- Incorporate the possibility of extreme events, even those never seen before?\n- Update beliefs in real-time as market conditions evolved?\n- Quantify uncertainty in a way that directly informed position sizing?\n\nWelcome to Bayesian thinking for commodities trading.\n\n## Learning Objectives\n\nBy the end of this module, you will be able to:\n1. Apply Bayes' theorem to update trading beliefs based on new market data\n2. Distinguish between Bayesian and Frequentist approaches in financial contexts\n3. Construct prior distributions that encode market knowledge\n4. Calculate posterior probabilities for trading decisions\n5. Interpret probability as degree of belief about future price movements

## 1.1 What is Bayesian Thinking?\n\n### The Frequentist Trader vs The Bayesian Trader\n\n**Frequentist Trader:**\n- \"Based on the last 1000 days, oil has closed higher 52% of the time\"\n- \"The 95% confidence interval for tomorrow's price is $70-75\"\n- \"We need more data to be statistically significant\"\n\n**Bayesian Trader:**\n- \"Given today's inventory report, I update my belief that oil will rise to 68%\"\n- \"There's a 95% probability tomorrow's price is between $70-75\"\n- \"I can start with expert knowledge and update as data arrives\"

In [ ]:
# Import required libraries\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom scipy import stats\nimport yfinance as yf\nfrom datetime import datetime, timedelta\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# Set style for better visualizations\nsns.set_style('whitegrid')\nplt.rcParams['figure.figsize'] = (12, 6)\nplt.rcParams['font.size'] = 11\n\n# Set random seed for reproducibility\nnp.random.seed(42)\n\nprint(\"Libraries loaded successfully!\")\nprint(f\"NumPy version: {np.__version__}\")\nprint(f\"Pandas version: {pd.__version__}\")

## 1.2 Bayes' Theorem: The Foundation\n\n### The Formula That Changes Everything\n\n$$P(H|D) = \\frac{P(D|H) \\times P(H)}{P(D)}$$\n\nWhere:\n- $P(H|D)$ = **Posterior**: Probability of hypothesis given data (what we want)\n- $P(D|H)$ = **Likelihood**: Probability of seeing data given hypothesis\n- $P(H)$ = **Prior**: Initial belief about hypothesis\n- $P(D)$ = **Evidence**: Total probability of seeing the data\n\n### Trading Translation\n\n$$P(\\text{Uptrend}|\\text{Positive News}) = \\frac{P(\\text{Positive News}|\\text{Uptrend}) \\times P(\\text{Uptrend})}{P(\\text{Positive News})}$$

In [ ]:
def bayes_theorem(prior, likelihood, evidence):\n    \"\"\"\n    Calculate posterior probability using Bayes' theorem\n    \n    Parameters:\n    -----------\n    prior : float\n        Prior probability P(H)\n    likelihood : float\n        Likelihood P(D|H)\n    evidence : float\n        Evidence P(D)\n    \n    Returns:\n    --------\n    float : Posterior probability P(H|D)\n    \"\"\"\n    posterior = (likelihood * prior) / evidence\n    return posterior\n\n# Trading Example: Oil Price Direction\n# Hypothesis: Oil will rise tomorrow\n\n# Prior: Historical probability oil rises (any given day)\nprior_rise = 0.52\n\n# New Data: Inventory report shows unexpected draw (bullish signal)\n# Likelihood: Probability of inventory draw given oil will rise\nlikelihood_draw_given_rise = 0.75\n\n# Probability of inventory draw given oil will fall\nlikelihood_draw_given_fall = 0.30\n\n# Evidence: Total probability of seeing inventory draw\nevidence_draw = (likelihood_draw_given_rise * prior_rise + \n                likelihood_draw_given_fall * (1 - prior_rise))\n\n# Calculate posterior\nposterior_rise = bayes_theorem(prior_rise, likelihood_draw_given_rise, evidence_draw)\n\nprint(\"Bayesian Oil Price Prediction:\")\nprint(\"=\" * 40)\nprint(f\"Prior belief (oil rises): {prior_rise:.1%}\")\nprint(f\"New data: Unexpected inventory draw\")\nprint(f\"Updated belief (oil rises): {posterior_rise:.1%}\")\nprint(f\"Change in conviction: {(posterior_rise - prior_rise):.1%}\")

## 1.3 Interactive Bayesian Updating\n\nLet's visualize how beliefs update with successive pieces of information:

In [ ]:
class BayesianTradingSignal:\n    \"\"\"\n    A class to demonstrate sequential Bayesian updating for trading signals\n    \"\"\"\n    \n    def __init__(self, prior=0.5):\n        self.prior = prior\n        self.posterior = prior\n        self.history = [prior]\n        self.signals = []\n    \n    def update(self, signal_name, likelihood_bull, likelihood_bear):\n        \"\"\"\n        Update belief based on new signal\n        \"\"\"\n        # Calculate evidence\n        evidence = (likelihood_bull * self.posterior + \n                   likelihood_bear * (1 - self.posterior))\n        \n        # Update posterior\n        self.posterior = (likelihood_bull * self.posterior) / evidence\n        \n        # Store history\n        self.history.append(self.posterior)\n        self.signals.append(signal_name)\n        \n        return self.posterior\n    \n    def plot_evolution(self):\n        \"\"\"\n        Visualize how belief evolved with each signal\n        \"\"\"\n        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))\n        \n        # Plot 1: Belief evolution\n        ax1.plot(self.history, 'b-', linewidth=2, marker='o', markersize=8)\n        ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)\n        ax1.axhline(y=self.prior, color='green', linestyle='--', \n                   alpha=0.5, label=f'Initial Prior: {self.prior:.2f}')\n        ax1.set_xlabel('Signal Number')\n        ax1.set_ylabel('P(Bullish Market)')\n        ax1.set_title('Bayesian Belief Evolution')\n        ax1.set_ylim([0, 1])\n        ax1.grid(True, alpha=0.3)\n        ax1.legend()\n        \n        # Add signal labels\n        for i, signal in enumerate(self.signals):\n            ax1.annotate(signal, (i+1, self.history[i+1]), \n                        textcoords=\"offset points\", \n                        xytext=(0,10), ha='center', fontsize=8)\n        \n        # Plot 2: Final belief distribution\n        x = np.linspace(0, 1, 1000)\n        # Use Beta distribution to represent uncertainty\n        a = self.posterior * 100  # Scale for visualization\n        b = (1 - self.posterior) * 100\n        y = stats.beta.pdf(x, a, b)\n        \n        ax2.fill_between(x, y, alpha=0.3, color='blue')\n        ax2.plot(x, y, 'b-', linewidth=2)\n        ax2.axvline(x=self.posterior, color='red', linestyle='--', \n                   label=f'Final Belief: {self.posterior:.3f}')\n        ax2.set_xlabel('P(Bullish Market)')\n        ax2.set_ylabel('Probability Density')\n        ax2.set_title('Final Belief Distribution')\n        ax2.legend()\n        ax2.grid(True, alpha=0.3)\n        \n        plt.tight_layout()\n        plt.show()\n\n# Example: Morning trading session with multiple signals\ntrader = BayesianTradingSignal(prior=0.5)  # Start neutral\n\n# Signal 1: Positive overnight Asian markets\ntrader.update(\"Asia +1.2%\", likelihood_bull=0.65, likelihood_bear=0.35)\nprint(f\"After Asian markets: {trader.posterior:.3f}\")\n\n# Signal 2: Oil inventory draw\ntrader.update(\"Inventory -3M\", likelihood_bull=0.70, likelihood_bear=0.25)\nprint(f\"After inventory data: {trader.posterior:.3f}\")\n\n# Signal 3: USD weakens\ntrader.update(\"USD -0.5%\", likelihood_bull=0.60, likelihood_bear=0.40)\nprint(f\"After USD move: {trader.posterior:.3f}\")\n\n# Signal 4: Technical breakout\ntrader.update(\"Break $72\", likelihood_bull=0.75, likelihood_bear=0.30)\nprint(f\"After technical breakout: {trader.posterior:.3f}\")\n\n# Visualize the evolution\ntrader.plot_evolution()

## 1.4 Real Market Application: WTI Crude Oil\n\nLet's apply Bayesian thinking to actual crude oil data:

In [ ]:
# Download recent WTI crude oil data\ndef get_oil_data():\n    \"\"\"Download WTI crude oil futures data\"\"\"\n    try:\n        # Download WTI crude oil futures\n        oil = yf.download('CL=F', start='2022-01-01', \n                         end=datetime.now(), progress=False)\n        oil['Returns'] = oil['Close'].pct_change()\n        oil['Direction'] = (oil['Returns'] > 0).astype(int)\n        return oil\n    except Exception as e:\n        print(f\"Error downloading data: {e}\")\n        # Fallback to synthetic data for demonstration\n        dates = pd.date_range(start='2022-01-01', end='2024-01-01', freq='D')\n        prices = 70 + np.cumsum(np.random.randn(len(dates)) * 2)\n        oil = pd.DataFrame({'Close': prices}, index=dates)\n        oil['Returns'] = oil['Close'].pct_change()\n        oil['Direction'] = (oil['Returns'] > 0).astype(int)\n        return oil\n\noil_data = get_oil_data()\n\n# Calculate historical statistics\nup_days = oil_data['Direction'].sum()\ntotal_days = len(oil_data['Direction'].dropna())\nhistorical_up_prob = up_days / total_days\n\nprint(\"WTI Crude Oil Historical Analysis\")\nprint(\"=\" * 40)\nprint(f\"Data period: {oil_data.index[0].date()} to {oil_data.index[-1].date()}\")\nprint(f\"Total trading days: {total_days}\")\nprint(f\"Up days: {up_days} ({historical_up_prob:.1%})\")\nprint(f\"Down days: {total_days - up_days} ({1-historical_up_prob:.1%})\")\nprint(f\"\\nCurrent price: ${oil_data['Close'].iloc[-1]:.2f}\")\nprint(f\"Average daily move: {oil_data['Returns'].std() * 100:.2f}%\")

## 1.5 Building a Bayesian Trading Model

In [ ]:
def bayesian_momentum_signal(data, lookback=20):\n    \"\"\"\n    Create a Bayesian momentum signal that updates beliefs based on recent performance\n    \"\"\"\n    signals = []\n    beliefs = []\n    \n    # Start with historical prior\n    prior = 0.52  # Slight upward bias for commodities\n    \n    for i in range(lookback, len(data)):\n        # Get recent returns\n        recent_returns = data['Returns'].iloc[i-lookback:i]\n        \n        # Calculate momentum signal\n        positive_days = (recent_returns > 0).sum()\n        momentum_strength = positive_days / lookback\n        \n        # Likelihood of seeing this momentum if market is bullish/bearish\n        likelihood_bull = min(0.9, 0.3 + momentum_strength * 0.7)\n        likelihood_bear = max(0.1, 0.7 - momentum_strength * 0.7)\n        \n        # Calculate evidence\n        evidence = likelihood_bull * prior + likelihood_bear * (1 - prior)\n        \n        # Update belief (posterior)\n        if evidence > 0:\n            posterior = (likelihood_bull * prior) / evidence\n        else:\n            posterior = prior\n        \n        # Trading signal: 1 if bullish (>0.6), -1 if bearish (<0.4), 0 otherwise\n        if posterior > 0.6:\n            signal = 1\n        elif posterior < 0.4:\n            signal = -1\n        else:\n            signal = 0\n        \n        signals.append(signal)\n        beliefs.append(posterior)\n        \n        # Update prior for next iteration (optional: can use fixed prior)\n        prior = 0.8 * prior + 0.2 * posterior  # Smooth updating\n    \n    return signals, beliefs\n\n# Apply to oil data\nsignals, beliefs = bayesian_momentum_signal(oil_data)\n\n# Add to dataframe\noil_data = oil_data.iloc[20:].copy()  # Align with signals\noil_data['Signal'] = signals\noil_data['Belief'] = beliefs\noil_data['Position'] = oil_data['Signal'].shift(1)  # Trade on next day\noil_data['Strategy_Return'] = oil_data['Position'] * oil_data['Returns']\n\n# Calculate performance\noil_data['Cumulative_Market'] = (1 + oil_data['Returns']).cumprod()\noil_data['Cumulative_Strategy'] = (1 + oil_data['Strategy_Return']).cumprod()\n\n# Plot results\nfig, axes = plt.subplots(3, 1, figsize=(14, 10))\n\n# Plot 1: Price and signals\nax1 = axes[0]\nax1.plot(oil_data.index, oil_data['Close'], 'k-', linewidth=1, label='WTI Price')\n\n# Mark buy/sell signals\nbuy_signals = oil_data[oil_data['Signal'] == 1]\nsell_signals = oil_data[oil_data['Signal'] == -1]\n\nax1.scatter(buy_signals.index, buy_signals['Close'], \n           color='green', marker='^', s=100, label='Buy Signal', alpha=0.7)\nax1.scatter(sell_signals.index, sell_signals['Close'], \n           color='red', marker='v', s=100, label='Sell Signal', alpha=0.7)\n\nax1.set_ylabel('Price ($)')\nax1.set_title('WTI Crude Oil Price with Bayesian Trading Signals')\nax1.legend()\nax1.grid(True, alpha=0.3)\n\n# Plot 2: Bayesian belief evolution\nax2 = axes[1]\nax2.plot(oil_data.index, oil_data['Belief'], 'b-', linewidth=2)\nax2.fill_between(oil_data.index, 0.5, oil_data['Belief'], \n                 where=(oil_data['Belief'] > 0.5), color='green', alpha=0.3)\nax2.fill_between(oil_data.index, oil_data['Belief'], 0.5, \n                 where=(oil_data['Belief'] < 0.5), color='red', alpha=0.3)\nax2.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)\nax2.axhline(y=0.6, color='green', linestyle=':', alpha=0.5, label='Buy Threshold')\nax2.axhline(y=0.4, color='red', linestyle=':', alpha=0.5, label='Sell Threshold')\nax2.set_ylabel('P(Bullish Market)')\nax2.set_title('Bayesian Belief Evolution')\nax2.set_ylim([0, 1])\nax2.legend()\nax2.grid(True, alpha=0.3)\n\n# Plot 3: Strategy performance\nax3 = axes[2]\nax3.plot(oil_data.index, oil_data['Cumulative_Market'], \n        'gray', linewidth=2, label='Buy & Hold')\nax3.plot(oil_data.index, oil_data['Cumulative_Strategy'], \n        'blue', linewidth=2, label='Bayesian Strategy')\nax3.set_ylabel('Cumulative Return')\nax3.set_xlabel('Date')\nax3.set_title('Strategy Performance Comparison')\nax3.legend()\nax3.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.show()\n\n# Performance metrics\ntotal_return_market = (oil_data['Cumulative_Market'].iloc[-1] - 1) * 100\ntotal_return_strategy = (oil_data['Cumulative_Strategy'].iloc[-1] - 1) * 100\nsharpe_market = oil_data['Returns'].mean() / oil_data['Returns'].std() * np.sqrt(252)\nsharpe_strategy = oil_data['Strategy_Return'].mean() / oil_data['Strategy_Return'].std() * np.sqrt(252)\n\nprint(\"\\nPerformance Summary\")\nprint(\"=\" * 40)\nprint(f\"Buy & Hold Return: {total_return_market:.2f}%\")\nprint(f\"Bayesian Strategy Return: {total_return_strategy:.2f}%\")\nprint(f\"Buy & Hold Sharpe: {sharpe_market:.3f}\")\nprint(f\"Bayesian Strategy Sharpe: {sharpe_strategy:.3f}\")\nprint(f\"Number of trades: {(oil_data['Signal'].diff() != 0).sum()}\")

## 1.6 Exercise Section\n\n### Exercise 1: Calculate Posterior Probability (Easy)\n\nA technical indicator correctly predicts oil price direction 70% of the time when oil rises, and 40% of the time when oil falls. Historically, oil rises 55% of days. The indicator is currently bullish. What is the probability oil will rise?

In [ ]:
# EXERCISE: Fill in the missing values\ndef calculate_indicator_posterior():\n    # Given information:\n    prior_rise = 0.55  # P(rise)\n    likelihood_bullish_given_rise = 0.70  # P(bullish|rise)\n    likelihood_bullish_given_fall = 0.40  # P(bullish|fall)\n    \n    # TODO: Calculate the evidence P(bullish)\n    evidence = _______  # Fill this in\n    \n    # TODO: Calculate posterior P(rise|bullish)\n    posterior = _______  # Fill this in\n    \n    return posterior\n\n# Test your answer\n# posterior_result = calculate_indicator_posterior()\n# print(f\"Probability oil rises given bullish signal: {posterior_result:.2%}\")

In [ ]:
# SOLUTION (Hidden by default)\ndef calculate_indicator_posterior_solution():\n    prior_rise = 0.55\n    likelihood_bullish_given_rise = 0.70\n    likelihood_bullish_given_fall = 0.40\n    \n    # Calculate evidence\n    evidence = (likelihood_bullish_given_rise * prior_rise + \n               likelihood_bullish_given_fall * (1 - prior_rise))\n    \n    # Calculate posterior\n    posterior = (likelihood_bullish_given_rise * prior_rise) / evidence\n    \n    return posterior\n\n# Verify solution\nresult = calculate_indicator_posterior_solution()\nprint(f\"Solution: Probability oil rises = {result:.2%}\")\nprint(f\"The bullish signal increases our belief from 55% to {result:.1%}\")

### Exercise 2: Sequential Updating (Medium)\n\nImplement a function that sequentially updates beliefs given multiple signals:

In [ ]:
# EXERCISE: Complete the sequential update function\ndef sequential_bayesian_update(prior, signals):\n    \"\"\"\n    Sequentially update belief given multiple signals\n    \n    Parameters:\n    prior: Initial belief P(bull market)\n    signals: List of tuples (likelihood_bull, likelihood_bear)\n    \n    Returns:\n    List of posterior beliefs after each signal\n    \"\"\"\n    posteriors = []\n    current_belief = prior\n    \n    for likelihood_bull, likelihood_bear in signals:\n        # TODO: Calculate evidence\n        evidence = _______\n        \n        # TODO: Update belief\n        current_belief = _______\n        \n        posteriors.append(current_belief)\n    \n    return posteriors\n\n# Test data\ntest_signals = [\n    (0.7, 0.3),  # Strong bullish signal\n    (0.6, 0.4),  # Weak bullish signal\n    (0.3, 0.7),  # Strong bearish signal\n]\n\n# results = sequential_bayesian_update(0.5, test_signals)\n# print(\"Belief evolution:\", results)

## Module 1 Quiz\n\nTest your understanding with these questions:\n\n1. **What is the key philosophical difference between Bayesian and Frequentist approaches?**\n   - A) Bayesians use more data\n   - B) Bayesians interpret probability as degree of belief\n   - C) Frequentists don't use probability\n   - D) There is no real difference\n\n2. **In Bayes' theorem, what does the likelihood represent?**\n   - A) Our initial belief\n   - B) The final answer\n   - C) How probable the data is given our hypothesis\n   - D) The total probability\n\n3. **When should you use an informative prior?**\n   - A) Never, always be objective\n   - B) When you have relevant domain expertise or historical data\n   - C) Only with small datasets\n   - D) Only for commodities trading\n\n4. **What happens to the posterior as you get more data?**\n   - A) It becomes less certain\n   - B) It converges toward the truth regardless of prior\n   - C) It stays the same\n   - D) It always increases\n\n5. **Why is Bayesian thinking valuable for trading?**\n   - A) It guarantees profits\n   - B) It provides a framework for updating beliefs with new information\n   - C) It eliminates all uncertainty\n   - D) It requires less data

## Common Pitfalls and How to Avoid Them\n\n### Pitfall 1: Using Improper Priors\n**Mistake**: Setting prior to 0 or 1 (absolute certainty)\n**Why it's bad**: No amount of data can change your belief\n**Fix**: Use weak priors (e.g., 0.1 to 0.9) unless truly certain\n\n### Pitfall 2: Double-Counting Evidence\n**Mistake**: Using correlated signals as independent\n**Why it's bad**: Overconfidence in posterior\n**Fix**: Model correlation explicitly or use only independent signals\n\n### Pitfall 3: Ignoring Base Rates\n**Mistake**: Focusing only on likelihood, ignoring prior\n**Why it's bad**: Rare events get overweighted\n**Fix**: Always consider base rates (priors) in analysis\n\n### Pitfall 4: Not Updating Sequentially\n**Mistake**: Throwing away yesterday's posterior\n**Why it's bad**: Loses information\n**Fix**: Yesterday's posterior becomes today's prior\n\n## Module Summary Checklist\n\n✓ Understand Bayes' theorem components (prior, likelihood, posterior, evidence)\n✓ Can calculate posterior probabilities by hand\n✓ Know when to use Bayesian vs Frequentist approaches\n✓ Can implement sequential Bayesian updating\n✓ Applied Bayesian thinking to real oil price data\n✓ Built a simple Bayesian trading strategy\n✓ Understand common pitfalls and how to avoid them\n\n## Next Module Preview\n\nIn Module 2, we'll dive deeper into **prior selection and elicitation**. You'll learn:\n- How to encode market expertise into mathematical priors\n- Conjugate priors for computational efficiency\n- Prior predictive checks to validate your assumptions\n- Hierarchical priors for modeling market regimes\n\nWe'll apply these concepts to model seasonal patterns in agricultural commodities and encode trader intuition into quantitative models.